In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import concurrent.futures as cf
import multiprocessing as mp

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [10]:
site = 'https://datascience.stackexchange.com/questions?tab=Votes&pagesize=50'
noofpages = None
response = requests.get(site)
soup = BeautifulSoup(response.text, 'html.parser')
noofpages = int(soup.find('div', class_= 's-pagination site557 themed pager float-left').find_all('a')[-2].text)

In [11]:
print(noofpages)

727


In [12]:
urls = []
base_url = 'https://datascience.stackexchange.com/questions?tab=Votes&pagesize=50'
for i in range(1, int(noofpages) + 1):
        if i == 1:
            urls.append(base_url)
        else:
            urls.append(base_url + '&page=' + str(i))

In [13]:
def scrape():
    question_links = []
    for url in tqdm(urls):
        page = requests.get(url,headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        for link in soup.find('div', attrs= {'class':'flush-left','id':'questions'}).find_all('a', attrs = {'class':'s-link'}):
            question_links.append(link.get('href'))
    return pd.DataFrame(question_links,columns=['Question Links'])

df = scrape()
df.to_csv('question_links.csv',index=False)

100%|██████████| 727/727 [11:22<00:00,  1.07it/s]


def scrape_page(url):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    return [link.get('href') for link in soup.find('div', attrs={'class': 'flush-left', 'id': 'questions'}).find_all('a', attrs={'class': 's-link'})]

def scrape():
    question_links = []    
    with cf.ThreadPoolExecutor(max_workers=2) as executor:
        # Submit scraping tasks to the ThreadPoolExecutor
        future_to_url = {executor.submit(scrape_page, url): url for url in urls}
        
        # Process the completed tasks
        for future in cf.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                question_links.extend('https://stats.stackexchange.com' + link for link in result)
            except Exception as e:
                print(f"Error while scraping {url}: {e}")

    return pd.DataFrame(question_links, columns=['Question Links'])
